# Imports

In [1]:
import pandas as pd
import re
import os


# lectura

# funciones

In [2]:
# 

root_excels = "C:\\Users\\hsakurada\\Desktop\\ESTUDIO\\wea\\CMS-PRUEBA-TECNICA\\descargas"

MESES = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4,
    "mayo": 5, "junio": 6, "julio": 7, "agosto": 8,
    "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}
años = ['2025','2024','2023','2022','2021','2020','2019','2018','2017','2016','2015','2014','2013','2012','2011','2010']

In [3]:

def extraer_mes_nomb_excel(nombre_archivo):
    # quitar extensión
    nombre = os.path.splitext(nombre_archivo)[0].lower()

    # separar por espacios, guiones, underscores, etc.
    partes = re.split(r"[ _\-]+", nombre)

    # buscar coincidencia exacta
    for parte in partes:
        if parte in MESES:
            return parte  # retorna el nombre del mes encontrado

    # búsqueda más flexible: " - diciembre 2012"
    for mes in MESES:
        if mes in nombre:
            return mes

    return None


def extraer_año(nombre_archivo):
    
    # quitar extensión
    nombre = os.path.splitext(nombre_archivo)[0].lower()

    # separar por espacios, guiones, underscores, etc.
    partes = re.split(r"[ _\-]+", nombre)

    # buscar coincidencia exacta
    for parte in partes:
        if parte in años:
            return parte  # retorna el año encontrado

    # búsqueda más flexible: " - diciembre 2012"
    for año in años:
        if año in nombre:
            return año

    return None

import os

def listar_todos_los_root_excels(root_excels):
    excels = []
    for root, dirs, files in os.walk(root_excels):
        for file in files:
            if file.lower().endswith(('.xlsx', '.xls')):
                excels.append(os.path.join(root, file))
    return excels



In [4]:
rutas = listar_todos_los_root_excels(root_excels)
rutas[:4]

['C:\\Users\\hsakurada\\Desktop\\ESTUDIO\\wea\\CMS-PRUEBA-TECNICA\\descargas\\2020\\Base de Datos de Transporte Público - Abril 2021.xlsx',
 'C:\\Users\\hsakurada\\Desktop\\ESTUDIO\\wea\\CMS-PRUEBA-TECNICA\\descargas\\2020\\Base de Datos de Transporte Público - Agosto 2021.xlsx',
 'C:\\Users\\hsakurada\\Desktop\\ESTUDIO\\wea\\CMS-PRUEBA-TECNICA\\descargas\\2020\\Base de Datos de Transporte Público - Diciembre 2021.xlsx',
 'C:\\Users\\hsakurada\\Desktop\\ESTUDIO\\wea\\CMS-PRUEBA-TECNICA\\descargas\\2020\\Base de Datos de Transporte Público - Enero 2021.xlsx']

In [5]:
import os
import re
import unicodedata
import pandas as pd
from rapidfuzz import fuzz, process as fuzzprocess

# ============================================================
# PRECOMPILAR REGEX (20x más rápido)
# ============================================================
_re_non_alnum = re.compile(r"[^0-9a-z]+")
_re_multi_unders = re.compile(r"_+")

# ============================================================
# NORMALIZACIÓN RÁPIDA
# ============================================================
def normalize_fast(text: str) -> str:
    """Normaliza texto MUY rápido: sin acentos, minúsculas, underscores limpios."""
    if not isinstance(text, str):
        return text

    text = unicodedata.normalize("NFKD", text)
    text = "".join(c for c in text if not unicodedata.combining(c))
    text = text.lower()

    text = _re_non_alnum.sub("_", text)
    text = _re_multi_unders.sub("_", text)
    return text.strip("_")


def normalize_sheet_name(name):
    """Normaliza y singulariza para evitar 'vehiculos' / 'vehiculo'."""
    norm = normalize_fast(name)
    # singularización simple
    if norm.endswith("es") and len(norm) > 3:
        norm = norm[:-2]
    elif norm.endswith("s") and len(norm) > 2:
        norm = norm[:-1]
    return norm


def normalize_column_name(name):
    return normalize_fast(name)


# ============================================================
# UNIFICADOR DE COLUMNAS (OPTIMIZADO)
# ============================================================
def unify_column_names_fast(dfs, cutoff=0.75):
    """
    Unifica nombres de columnas con:
    - normalización
    - bucketing por prefijo (reduce fuzzy match 80%)
    """

    # --------------- recolectar columnas normalizadas ---------------
    col_master = set()
    normalized_per_df = []

    for df in dfs:
        norm_cols = [normalize_column_name(c) for c in df.columns]
        normalized_per_df.append(norm_cols)
        col_master.update(norm_cols)

    col_master = list(col_master)

    # --------------- buckets por primeros 4 caracteres ---------------
    buckets = {}
    for col in col_master:
        key = col[:4]
        buckets.setdefault(key, []).append(col)

    # --------------- renombrar DataFrames ---------------
    results = []
    for df, norm_cols in zip(dfs, normalized_per_df):
        rename_map = {}
        for orig, norm in zip(df.columns, norm_cols):
            candidates = buckets.get(norm[:4], [])

            if norm in candidates:
                rename_map[orig] = norm
                continue

            # fuzzy solo cuando necesario
            best, score = fuzzprocess.extractOne(norm, candidates, scorer=fuzz.ratio) or (None, 0)
            rename_map[orig] = best if score >= cutoff * 100 else norm

        results.append(df.rename(columns=rename_map))

    return results


# ============================================================
# AGRUPADOR DE HOJAS SIMILARES (vehiculo / vehiculos / vehículo)
# ============================================================
def agrupar_nombres_hoja(sheet_names, cutoff=80):
    """
    Une nombres similares en un solo grupo:
    Ej: vehiculo, vehiculos, vehículo → vehiculo
    """
    grupos = {}
    canonicos = []

    for name in sheet_names:
        norm = normalize_sheet_name(name)

        if canonicos:
            match = fuzzprocess.extractOne(norm, canonicos, scorer=fuzz.ratio)
            if match and match[1] >= cutoff:
                grupos[match[0]].append(name)
                continue

        grupos[norm] = [name]
        canonicos.append(norm)

    return grupos



In [6]:
def escanear_hojas(lista_rutas, verbose=True):
    all_sheet_names = []
    file_sheets = {}

    for ruta in lista_rutas:
        try:
            xls = pd.ExcelFile(ruta)
            nombres = xls.sheet_names
            file_sheets[ruta] = nombres
            all_sheet_names.extend(nombres)

            if verbose:
                print(f"📄 {os.path.basename(ruta)}")
                print(f"   → Hojas: {nombres}")

        except Exception as e:
            print(f"❌ Error leyendo {ruta}: {e}")
            continue

    return file_sheets, all_sheet_names


In [7]:
def obtener_grupo_final(nombre_hoja, hoja_norm, equivalencias):
    for canon, miembros in equivalencias.items():
        if nombre_hoja in miembros or hoja_norm == canon:
            return canon
    return hoja_norm  # si no encuentra grupo


In [8]:
def agrupar_hojas(all_sheet_names, cutoff=80, verbose=True):
    equivalencias = agrupar_nombres_hoja(all_sheet_names, cutoff=cutoff)

    if verbose:
        print("\n📌 Grupos detectados:")
        for canon, items in equivalencias.items():
            print(f"   • {canon}: {items}")

    return equivalencias


In [9]:
def previsualizar_columnas(file_sheets, equivalencias, n_rows_preview=5, verbose=True):
    columnas_por_grupo = {}

    for ruta, hojas in file_sheets.items():
        try:
            xls = pd.ExcelFile(ruta)
        except:
            continue

        for hoja_original in hojas:
            hoja_norm = normalize_sheet_name(hoja_original)
            grupo = obtener_grupo_final(hoja_original, hoja_norm, equivalencias)

            try:
                df_preview = xls.parse(hoja_original, nrows=n_rows_preview)
            except:
                continue

            columnas = list(df_preview.columns)

            columnas_por_grupo.setdefault(grupo, [])
            columnas_por_grupo[grupo].append({
                "archivo": os.path.basename(ruta),
                "hoja": hoja_original,
                "columnas": columnas
            })

    if verbose:
        print("\n🔎 PREVISUALIZACIÓN DE COLUMNAS POR GRUPO")
        for grupo, items in columnas_por_grupo.items():
            print(f"\n=== Grupo: {grupo} ===")
            for entry in items:
                print(f"  • Archivo: {entry['archivo']}")
                print(f"    Hoja:    {entry['hoja']}")
                print(f"    Columnas: {entry['columnas']}")

    return columnas_por_grupo


In [10]:
def obtener_grupo_final(nombre_hoja, hoja_norm, equivalencias):
    for canon, miembros in equivalencias.items():
        if nombre_hoja in miembros or hoja_norm == canon:
            return canon
    return hoja_norm


In [11]:
def procesar_archivos(file_sheets, equivalencias, verbose=True):
    grupos = {}

    for ruta, hojas in file_sheets.items():
        nombre_archivo = os.path.basename(ruta)
        mes = extraer_mes_nomb_excel(nombre_archivo)
        año = extraer_año(nombre_archivo)

        if verbose:
            print(f"\n📂 Procesando {nombre_archivo}")

        xls = pd.ExcelFile(ruta)

        for hoja_original in hojas:
            hoja_norm = normalize_sheet_name(hoja_original)
            hoja_final = obtener_grupo_final(hoja_original, hoja_norm, equivalencias)

            if verbose:
                print(f"   📝 {hoja_original}  → grupo={hoja_final}")

            try:
                df = xls.parse(hoja_original)
            except Exception as e:
                print(f"   ❌ Error leyendo {hoja_original}: {e}")
                continue

            df["mes"] = mes
            df["año"] = año

            grupos.setdefault(hoja_final, []).append(df)

    return grupos


In [12]:
# 1) Escanear nombres de hojas
file_sheets, all_sheet_names = escanear_hojas(rutas)

# 2) Agrupar por fuzzy
equivalencias = agrupar_hojas(all_sheet_names)




📄 Base de Datos de Transporte Público - Abril 2021.xlsx
   → Hojas: ['Servicios', 'Replegal', 'Vehículos', 'O-D Recorridos', 'Trazados']
📄 Base de Datos de Transporte Público - Agosto 2021.xlsx
   → Hojas: ['Servicios', 'RepLegales', 'Vehículo', 'O-D Recorrido', 'Trazados']
📄 Base de Datos de Transporte Público - Diciembre 2021.xlsx
   → Hojas: ['Servicios', 'RepLegales', 'Vehículo', 'O-D Recorridos', 'Trazado']
📄 Base de Datos de Transporte Público - Enero 2021.xlsx
   → Hojas: ['Servicios', 'RepLegales', 'Vehiculos', 'O-D Recorridos', 'Trazados']
📄 Base de Datos de Transporte Público - Febrero 2021.xlsx
   → Hojas: ['Servicio', 'RepLegales', 'Vehículo', 'O-D Recorrido', 'Trazado']
📄 Base de Datos de Transporte Público - Julio 2021.xlsx
   → Hojas: ['Servicios', 'Representantes Legales', 'Vehículos', 'O-D Recorridos', 'Trazados']
📄 Base de Datos de Transporte Público - Junio 2021.xlsx
   → Hojas: ['Servicios', 'RepLegales', 'Vehiculos', 'O-D Recorridos', 'Trazados ']
📄 Base de Datos d

In [ ]:
# 3) Previsualizar las columnas por grupo (VERIFICACIÓN MANUAL)
columnas_preview = previsualizar_columnas(file_sheets, equivalencias)

columnas_preview


🔎 PREVISUALIZACIÓN DE COLUMNAS POR GRUPO

=== Grupo: servicio ===
  • Archivo: Base de Datos de Transporte Público - Abril 2021.xlsx
    Hoja:    Servicios
    Columnas: ['REGION', 'FOLIO', 'TIPO_SERVICIO', 'FLOTA VIGENTE', 'NRO_LINEA', 'RUT_RESPONSABLE', 'NOMBRE_RESPONSABLE', 'NOMBRE_FANTASIA', 'TIPO_PERSONA', 'DOMICILIO', 'COMUNA', 'REGION.1', 'TELEFONO', 'FAX', 'EMAIL']
  • Archivo: Base de Datos de Transporte Público - Agosto 2021.xlsx
    Hoja:    Servicios
    Columnas: ['REGION', 'FOLIO', 'TIPO_SERVICIO', 'FLOTA_VIGENTE ', 'NRO_LINEA', 'RUT_RESPONSABLE', 'NOMBRE_RESPONSABLE', 'NOMBRE_FANTASIA', 'TIPO_PERSONA', 'DOMICILIO', 'COMUNA', 'REGION.1', 'TELEFONO', 'FAX', 'EMAIL']
  • Archivo: Base de Datos de Transporte Público - Diciembre 2021.xlsx
    Hoja:    Servicios
    Columnas: ['REGION', 'FOLIO', 'TIPO_SERVICIO', 'FLOTA_VIGENTE', 'NRO_LINEA', 'RUT_RESPONSABLE', 'NOMBRE_RESPONSABLE', 'NOMBRE_FANTASIA', 'TIPO_PERSONA', 'DOMICILIO', 'COMUNA', 'REGION.1', 'TELEFONO', 'FAX', 'EMAIL

{'servicio': [{'archivo': 'Base de Datos de Transporte Público - Abril 2021.xlsx',
   'hoja': 'Servicios',
   'columnas': ['REGION',
    'FOLIO',
    'TIPO_SERVICIO',
    'FLOTA VIGENTE',
    'NRO_LINEA',
    'RUT_RESPONSABLE',
    'NOMBRE_RESPONSABLE',
    'NOMBRE_FANTASIA',
    'TIPO_PERSONA',
    'DOMICILIO',
    'COMUNA',
    'REGION.1',
    'TELEFONO',
    'FAX',
    'EMAIL']},
  {'archivo': 'Base de Datos de Transporte Público - Agosto 2021.xlsx',
   'hoja': 'Servicios',
   'columnas': ['REGION',
    'FOLIO',
    'TIPO_SERVICIO',
    'FLOTA_VIGENTE ',
    'NRO_LINEA',
    'RUT_RESPONSABLE',
    'NOMBRE_RESPONSABLE',
    'NOMBRE_FANTASIA',
    'TIPO_PERSONA',
    'DOMICILIO',
    'COMUNA',
    'REGION.1',
    'TELEFONO',
    'FAX',
    'EMAIL']},
  {'archivo': 'Base de Datos de Transporte Público - Diciembre 2021.xlsx',
   'hoja': 'Servicios',
   'columnas': ['REGION',
    'FOLIO',
    'TIPO_SERVICIO',
    'FLOTA_VIGENTE',
    'NRO_LINEA',
    'RUT_RESPONSABLE',
    'NOMBRE_RESPON

In [ ]:
grupos_df = procesar_archivos(file_sheets, equivalencias)

grupos_df



📂 Procesando Base de Datos de Transporte Público - Abril 2021.xlsx
   📝 Servicios  → grupo=servicio
   📝 Replegal  → grupo=replegal
   📝 Vehículos  → grupo=vehiculo
   📝 O-D Recorridos  → grupo=o_d_recorrido
   📝 Trazados  → grupo=trazado

📂 Procesando Base de Datos de Transporte Público - Agosto 2021.xlsx
   📝 Servicios  → grupo=servicio
   📝 RepLegales  → grupo=replegal
   📝 Vehículo  → grupo=vehiculo
   📝 O-D Recorrido  → grupo=o_d_recorrido
   📝 Trazados  → grupo=trazado

📂 Procesando Base de Datos de Transporte Público - Diciembre 2021.xlsx
   📝 Servicios  → grupo=servicio
   📝 RepLegales  → grupo=replegal
   📝 Vehículo  → grupo=vehiculo
   📝 O-D Recorridos  → grupo=o_d_recorrido
   📝 Trazado  → grupo=trazado

📂 Procesando Base de Datos de Transporte Público - Enero 2021.xlsx
   📝 Servicios  → grupo=servicio
   📝 RepLegales  → grupo=replegal
   📝 Vehiculos  → grupo=vehiculo
   📝 O-D Recorridos  → grupo=o_d_recorrido
   📝 Trazados  → grupo=trazado

📂 Procesando Base de Datos de Tr

{'servicio': [       REGION   FOLIO             TIPO_SERVICIO  FLOTA VIGENTE NRO_LINEA  \
  0           1      10      TAXI COLECTIVO RURAL              1       NaN   
  1           1    4608      TAXI COLECTIVO RURAL              1       NaN   
  2           1      27      TAXI COLECTIVO RURAL              1       NaN   
  3           1      67      TAXI COLECTIVO RURAL              1             
  4           1    4917      TAXI COLECTIVO RURAL              1       NaN   
  ...       ...     ...                       ...            ...       ...   
  41520      15    6074   MINIBUS RURAL CORRIENTE             14       NaN   
  41521      15    6111   MINIBUS RURAL CORRIENTE             15       NaN   
  41522      15    6143   MINIBUS RURAL CORRIENTE              4       NaN   
  41523      15    6155   MINIBUS RURAL CORRIENTE             10       NaN   
  41524      14  400031  MINIBUS URBANO CORRIENTE             11       NaN   
  
        RUT_RESPONSABLE                          

In [ ]:
grupos_df_unificados = {}

for grupo, dfs in grupos_df.items():
    print(f"\n👉 Normalizando columnas del grupo: {grupo}")
    dfs_norm = unify_column_names_fast(dfs)
    grupos_df_unificados[grupo] = dfs_norm



👉 Normalizando columnas del grupo: servicio

👉 Normalizando columnas del grupo: replegal

👉 Normalizando columnas del grupo: vehiculo

👉 Normalizando columnas del grupo: o_d_recorrido

👉 Normalizando columnas del grupo: trazado


In [ ]:
final_groups = {}

for grupo, dfs in grupos_df_unificados.items():
    final_groups[grupo] = pd.concat(dfs, ignore_index=True)


In [ ]:
final_groups

{'servicio':          region   folio         tipo_servicio  flota_vigente nro_linea  \
 0             1      10  TAXI COLECTIVO RURAL              1       NaN   
 1             1    4608  TAXI COLECTIVO RURAL              1       NaN   
 2             1      27  TAXI COLECTIVO RURAL              1       NaN   
 3             1      67  TAXI COLECTIVO RURAL              1             
 4             1    4917  TAXI COLECTIVO RURAL              1       NaN   
 ...         ...     ...                   ...            ...       ...   
 3167889      13  800013                   NaN            911       NaN   
 3167890      13  900015                   NaN           1339  UNIDAD 5   
 3167891      13  900012                   NaN           1435  UNIDAD 2   
 3167892      13  900013                   NaN           1460  UNIDAD 3   
 3167893      13  900021                   NaN           1613  UNIDAD 4   
 
         rut_responsable             nombre_responsable nombre_fantasia  \
 0         

In [ ]:
import os

output_folder = "parquets_finales"
os.makedirs(output_folder, exist_ok=True)

for nombre_grupo, df in final_groups.items():

    # Convertir TODAS las columnas object a string (evita 100% de problemas)
    df = df.copy()
    df = df.apply(lambda col: col.astype(str) if col.dtype == "object" else col)

    ruta = os.path.join(output_folder, f"{nombre_grupo}.parquet")
    df.to_parquet(ruta, index=False)
    print(f"✔ Guardado: {ruta}")


✔ Guardado: parquets_finales\servicio.parquet
✔ Guardado: parquets_finales\replegal.parquet
✔ Guardado: parquets_finales\vehiculo.parquet
✔ Guardado: parquets_finales\o_d_recorrido.parquet
✔ Guardado: parquets_finales\trazado.parquet
